In [1]:
import numpy as np
import pandas as pd
import os
import requests
import re
from nltk import FreqDist
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from cltk.stops.grc import STOPS as stopwords
from gensim.corpora import Dictionary
import unicodedata
import json

In [2]:
# find local version of the input dataset
local_paths = !find ~/Projects -name "LAGT_v3-0.parquet"
print(local_paths)

['/Users/vojtechkase/Projects/LAGT/data/large_files/LAGT_v3-0.parquet']


In [3]:
# load LAGT (v2.0) dataset locally or download it directly from Zenodo
try:
    LAGT = pd.read_parquet(local_paths[0])
except:
    pass
    #resp = requests.get("https://zenodo.org/record/7221150/files/LAGT_v2-0.json?download=1")
    # save it for next time
    #LAGT = pd.DataFrame(resp.json())
    #os.mkdir("../data/large_data/")
    #LAGT.to_json("../data/large_data/LIRE_v2-0.json")

# Short demonstration of the LAGT dataset...

In [4]:
# download and display first 5 rows of the dataset
#LAGT = pd.read_json("https://zenodo.org/record/7221150/files/LAGT_v2-0.json?download=1")
LAGT.head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,provenience,not_before,not_after,lemmatacount
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,"[[Πινυτός, ἀντιγράφω, θαυμάζω, ἀποδέχω, Διονύσ...",grecy,christian,101.0,200.0,34
8,tlg0005,tlg0005.tlg003,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77,1Kgr,"[[οὐδενός, εὐνητήρ], [μακροπτολέμοιο, μήτηρ, μ...",grecy,pagan,-400.0,-201.0,61
9,tlg0006,tlg0006.tlg020,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708,1Kgr,"[[φημί, γῆ, ἐκλείπω, πόλις, ξενοῦσθαι], [πάτρα...",grecy,pagan,-500.0,-401.0,10277
10,tlg0007,tlg0007.tlg146,tlg0007.tlg146.1st1K-grc1.xml,Plutarch,Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο,Οἴκοι τὰ Μιλήσια: ἐπὶ τῶν ὅποι μὴ προςήκει τὴν...,2685,1Kgr,"[[Μιλήσιος], [προςήκω, τρυφή, ἐπιδείκνυμι], [Ἀ...",grecy,pagan,1.0,200.0,1488
11,tlg0007,tlg0007.tlg147,tlg0007.tlg147.1st1K-grc1.xml,Plutarch,Ἐκλογὴ περὶ τῶν ἀδυνάτων,Κατὰ πετρῶν σπείρεις. Πλίνθον πλύνεις. Δικτύῳ ...,143,1Kgr,"[[πέτρα, σπείρω], [Πλίνθος, πλύνω, Δίκτυον, ἄν...",grecy,pagan,1.0,200.0,125


In [5]:
len(LAGT)

1710

In [6]:
LAGT["author_id"].nunique()

325

In [7]:

LAGT["wordcount"].sum()

32323612

In [8]:
LAGT[LAGT["author_id"].str.startswith("tlg0031")]

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,provenience,not_before,not_after,lemmatacount
1525,tlg0031a,tlg0031.tlg001,tlg0031.tlg001.perseus-grc2.xml,None,New Testament - Matthew,ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱοῦ Δαυεὶδ υἱ...,18288,perseus,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...",morphgnt,christian,1.0,100.0,8699
1526,tlg0031b,tlg0031.tlg002,tlg0031.tlg002.perseus-grc2.xml,None,New Testament - Mark,\n\n ΑΡΧΗ τοῦ εὐαγγελίου Ἰη...,11275,perseus,"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ...",morphgnt,christian,1.0,100.0,5308
1527,tlg0031luke,tlg0031.tlg003,tlg0031.tlg003.perseus-grc2.xml,Luke (the evangelist),New Testament - Luke,\n\n ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχε...,19459,perseus,"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...",morphgnt,christian,1.0,100.0,9194
1528,tlg0031john,tlg0031.tlg004,tlg0031.tlg004.perseus-grc2.xml,Johnannine literature,New Testament - John,"\n\n ΕΝ ΑΡΧΗ ἦν ὁ λόγος, κα...",15592,perseus,"[[ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θεός, ...",morphgnt,christian,1.0,100.0,6892
1529,tlg0031luke,tlg0031.tlg005,tlg0031.tlg005.perseus-grc2.xml,Luke (the evangelist),New Testament - Acts,\n τὸν μὲν πρῶτον λόγον ἐποιησάμην περὶ πάντων...,18405,perseus,"[[πρῶτος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἄρχω, Ἰ...",morphgnt,christian,1.0,100.0,9141
1530,tlg0031paul,tlg0031.tlg006,tlg0031.tlg006.perseus-grc2.xml,Paul of Tarsus,New Testament - Romans,\n\n ΠΑΥΛΟΣ δοῦλος Ἰησοῦ Χρ...,7107,perseus,"[[Παῦλος, δοῦλος, Χριστός, Ἰησοῦς, κλητός, ἀπό...",morphgnt,christian,1.0,100.0,3265
1531,tlg0031paul,tlg0031.tlg007,tlg0031.tlg007.perseus-grc2.xml,Paul of Tarsus,New Testament - 1 Corinthians,\n\n ΠΑΥΛΟΣ κλητὸς ἀπόστολο...,6814,perseus,"[[Παῦλος, κλητός, ἀπόστολος, Χριστός, Ἰησοῦς, ...",morphgnt,christian,1.0,100.0,3292
1532,tlg0031paul,tlg0031.tlg008,tlg0031.tlg008.perseus-grc2.xml,Paul of Tarsus,New Testament - 2 Corinthians,\n\n ΠΑΥΛΟΣ ἀπόστολος Χριστ...,4470,perseus,"[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ...",morphgnt,christian,1.0,100.0,1925
1533,tlg0031paul,tlg0031.tlg009,tlg0031.tlg009.perseus-grc2.xml,Paul of Tarsus,New Testament - Galatians,"\n\n ΠΑΥΛΟΣ ἀπόστολος, οὐκ ...",2235,perseus,"[[Παῦλος, ἀπόστολος, ἄνθρωπος, ἄνθρωπος, Ἰησοῦ...",morphgnt,christian,1.0,100.0,1058
1534,tlg0031pspa,tlg0031.tlg010,tlg0031.tlg010.perseus-grc2.xml,Pauline literature,New Testament - Ephesians,\n\n ΠΑΥΛΟΣ ἀπόστολος Χριστ...,2421,perseus,"[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ...",morphgnt,christian,1.0,100.0,1126


In [9]:
# extract a list of all lemmatized words from all lemmatized sentences
wordlist = [w for sent in [sent for work in LAGT["lemmatized_sentences"] for sent in work] for w in sent]
# count all instances of the word θεός
wordlist.count("θεός")

60033

In [10]:
# filter for texts from the fifth and fourth c. BCE
len(LAGT[(LAGT["not_before"].between(-500,-301)) | (LAGT["not_before"].between(-500,-301))])

443

In [11]:
# index for aristotle's nicomachean ethics
i = LAGT[LAGT["doc_id"]=="tlg0086.tlg010"].index[0]
LAGT.loc[i]

author_id                                                         tlg0086
doc_id                                                     tlg0086.tlg010
filename                                  tlg0086.tlg010.perseus-grc1.xml
author                                                          Aristotle
title                                                  Nicomachean Ethics
string                  πᾶσα τέχνη καὶ πᾶσα μέθοδος, ὁμοίως δὲ πρᾶξίς ...
wordcount                                                           56630
source                                                            perseus
lemmatized_sentences    [[πᾶς, τέχνη, πᾶς, μέθοδος, πρᾶξις, προαίρεσις...
lemmata_source                                                      glaux
provenience                                                         pagan
not_before                                                         -400.0
not_after                                                          -301.0
lemmatacount                          

# Explore overall vocabulary

In [12]:
sents = [sent for work in LAGT["lemmatized_sentences"] for sent in work]

In [13]:
len(sents)

2136459

In [14]:
min_freq = 10

def get_vocab(docs, min_freq=min_freq):
    words_flat = [item for sublist in docs for item in sublist]
    word_freq_tups = FreqDist(words_flat).most_common()
    vocabulary = [tup[0] for tup in word_freq_tups if tup[1] >= min_freq]
    #vocab_freqs = [len([doc for doc in docs if word in doc]) for word in vocabulary]
    return word_freq_tups, words_flat, vocabulary

In [15]:
word_freqs, words, vocabulary = get_vocab(sents)

In [16]:
pd.DataFrame(word_freqs[:20], columns=["lemma", "count"])

,lemma,count
0,οὗτος,329550
1,λέγω,239805
2,εἰμί,228068
3,αὐτός,213863
4,γίγνομαι,157466
5,ἔχω,146946
6,πολύς,121226
7,πᾶς,104593
8,φημί,99994
9,ἄλλος,96970


# Generate ngrams

In [17]:
doc_ids = LAGT["doc_id"].tolist()
len(doc_ids)

1710

In [18]:
#!mkdir ../data/large_data

In [19]:
line = 0
ids_lines = {}

f = open("../data/large_data/corpus_ngrams_bydocid_wide.txt", "w", encoding="utf-8")
for doc_id in doc_ids:
    lagt_subset = LAGT[LAGT["doc_id"]==doc_id]
    sents = [sen for work in lagt_subset["lemmatized_sentences"] for sen in work]
    sents_trigrams = [list(el) for sublist in [[ng for ng in nltk.trigrams(sent)] for sent in sents] for el in sublist]
    sents_fivegrams = [list(el) for sublist in [[ng for ng in nltk.ngrams(sent, n=5)] for sent in sents] for el in sublist]
    ngrams_data = sents + sents_fivegrams # + sents_trigrams
    ngrams_data = [" ".join(ngram) for ngram in ngrams_data]
    f.writelines("\n".join(ngrams_data)+"\n")
    if bool(ngrams_data):
        ids_lines[doc_id] = (line, line+len(ngrams_data))
        line += len(ngrams_data)
    else:
        ids_lines[doc_id] = (line, line+1)
        line += 1

In [20]:
pickle.dump(ids_lines, open("../data/ids_lines_wide.pickle", "wb"))

# Ngram example

In [21]:
# index for aristotle's nicomachean ethics
i = LAGT[LAGT["doc_id"]=="tlg0086.tlg010"].index[0]

In [22]:
i

1736

In [23]:
LAGT.loc[i]["string"].split("·")[0]

'πᾶσα τέχνη καὶ πᾶσα μέθοδος, ὁμοίως δὲ πρᾶξίς τε καὶ προαίρεσις, ἀγαθοῦ τινὸς ἐφίεσθαι δοκεῖ'

In [24]:
sent = LAGT.loc[i]["lemmatized_sentences"][0]
print(sent)

['πᾶς' 'τέχνη' 'πᾶς' 'μέθοδος' 'πρᾶξις' 'προαίρεσις' 'ἀγαθός' 'τις'
 'ἐφίημι' 'δοκέω']


In [25]:
trigrams = [list(ng) for ng in nltk.trigrams(sent)]
fivegrams = [list(ng) for ng in nltk.ngrams(sent, n=5)]
print([sent] + trigrams + fivegrams)

[array(['πᾶς', 'τέχνη', 'πᾶς', 'μέθοδος', 'πρᾶξις', 'προαίρεσις', 'ἀγαθός',
       'τις', 'ἐφίημι', 'δοκέω'], dtype=object), ['πᾶς', 'τέχνη', 'πᾶς'], ['τέχνη', 'πᾶς', 'μέθοδος'], ['πᾶς', 'μέθοδος', 'πρᾶξις'], ['μέθοδος', 'πρᾶξις', 'προαίρεσις'], ['πρᾶξις', 'προαίρεσις', 'ἀγαθός'], ['προαίρεσις', 'ἀγαθός', 'τις'], ['ἀγαθός', 'τις', 'ἐφίημι'], ['τις', 'ἐφίημι', 'δοκέω'], ['πᾶς', 'τέχνη', 'πᾶς', 'μέθοδος', 'πρᾶξις'], ['τέχνη', 'πᾶς', 'μέθοδος', 'πρᾶξις', 'προαίρεσις'], ['πᾶς', 'μέθοδος', 'πρᾶξις', 'προαίρεσις', 'ἀγαθός'], ['μέθοδος', 'πρᾶξις', 'προαίρεσις', 'ἀγαθός', 'τις'], ['πρᾶξις', 'προαίρεσις', 'ἀγαθός', 'τις', 'ἐφίημι'], ['προαίρεσις', 'ἀγαθός', 'τις', 'ἐφίημι', 'δοκέω']]


# Exploring subcorpora

In [26]:
periods = {
    "archaic" : (-800, -501),
    "classical" : (-500,-301),
    "hellenistic" : (-300,-1),
    "roman_peak" : (1, 200),
    "late_roman" : (201, 400)
}   

def get_periods(row):
    not_before = row["not_before"]
    not_after = row["not_after"]
    periods_covered = []
    try:
        for period_key in periods.keys():
            period = periods[period_key]
            if period[1] >= not_before and not_after >= period[0]:
                periods_covered.append(period_key)
    except:
        pass
    return periods_covered

In [27]:
LAGT["periods"] = LAGT.apply(get_periods, axis=1)

In [28]:
subcorpora_overview = []
for period_key in periods.keys():
    LAGT_subset = LAGT[LAGT["periods"].apply(lambda x: period_key in x)]
    subcorpora_overview.append(
        {"period" : period_key,
         "works_n" : len(LAGT_subset),
         "words_n" : LAGT_subset["wordcount"].sum()
         }
    )
        

In [29]:
subcorpora_overview_df = pd.DataFrame(subcorpora_overview)
subcorpora_overview_df

,period,works_n,words_n
0,archaic,59,338991
1,classical,458,4133038
2,hellenistic,238,4164793
3,roman_peak,752,14202575
4,late_roman,326,9294681


In [34]:
provenience_overview = LAGT.groupby("provenience").agg(
    works_n=('provenience', 'size'),
    words_n=('wordcount', 'sum')
)
provenience_overview

,works_n,words_n
provenience,,
christian,207,7574364
jewish,92,2103563
pagan,1345,20366621


### Overview by century

# Looking at frequencies for our manually annotated subset of words

In [95]:
translated_df = pd.read_json("../data/translated_df.json")
translated_df

,raw_translation,greek,english_full,english_minimal,category,greek_transliterated
0,Αἰθίοψ - Ethiopian,Αἰθίοψ,Ethiopian,Ethiopian,,Aithíops
1,Δαναοί - Danaans,Δαναοί,Danaans,Danaans,,Danaoí
2,Δημήτηρ - Demeter,Δημήτηρ,Demeter,Demeter,,Dēmḗtēr
3,Διόνυσος - Dionysus,Διόνυσος,Dionysus,Dionysus,,Diónusos
4,Εὐθύφρων - Euthyphron,Εὐθύφρων,Euthyphron,Euthyphron,,Euthúphrōn
...,...,...,...,...,...,...
501,Ὠκεανός - Oceanus,Ὠκεανός,Oceanus,Oceanus,,Ōkeanós
502,"ῥέζω - govern, manage",ῥέζω,"govern, manage",govern,,rhézō
503,"ῥητορική - rhetoric, art of speaking",ῥητορική,"rhetoric, art of speaking",rhetoric,,rhētorikḗ
504,"ῥῆσις - speech, word",ῥῆσις,"speech, word",speech,,rhêsis


In [100]:
subset = LAGT[LAGT["subcorpus"]=="archaic"]
sents = [sent for work in subset["lemmatized_sentences"] for sent in work]
word_freqs, words, vocabulary = get_vocab(sents)

In [101]:
dict(word_freqs)

{'εἰμί': 3428,
 'πᾶς': 1723,
 'ἀνήρ': 1634,
 'ἔρχομαι': 1623,
 'λέγω': 1506,
 'πολύς': 1499,
 'θεός': 1429,
 'ἔχω': 1412,
 'Ζεύς': 1322,
 'φημί': 1227,
 'μέγας': 1171,
 'ναῦς': 1129,
 'φίλος': 1005,
 'ὁράω': 936,
 'ὅδε': 915,
 'χείρ': 912,
 'θυμός': 899,
 'υἱός': 756,
 'κακός': 753,
 'γίγνομαι': 745,
 'Ἀχαιός': 740,
 'πατήρ': 729,
 'Ὀδυσσεύς': 725,
 'δίδωμι': 675,
 'Τρώς': 649,
 'παῖς': 626,
 'γῆ': 618,
 'αὐτός': 606,
 'οὗτος': 605,
 'ἄνθρωπος': 589,
 'ἐμός': 586,
 'φέρω': 580,
 'ἵππος': 575,
 'ἵστημι': 573,
 'τίθημι': 569,
 'ἀγαθός': 569,
 'φρήν': 559,
 'βάλλω': 554,
 'ἔπος': 548,
 'οἶδα': 546,
 'αἱρέω': 533,
 'μῦθος': 502,
 'πρῶτος': 492,
 'πόλις': 492,
 'βαίνω': 484,
 'ἐθέλω': 478,
 'καλός': 474,
 'ἄγω': 465,
 'Ἕκτωρ': 456,
 'ἑταῖρος': 452,
 'γυνή': 439,
 'τις': 439,
 'δῖος': 434,
 'ἔργον': 434,
 'πούς': 416,
 'Ἀχιλλεύς': 395,
 'ἀθάνατος': 389,
 'δῶμα': 387,
 'μήτηρ': 378,
 'ἑός': 370,
 'Ἀθήνη': 368,
 'ἱκνέομαι': 359,
 'λαός': 352,
 'λόγος': 349,
 'πόλεμος': 348,
 'ἄναξ': 345,
 'ἀκο

In [102]:
subcorpora = ['archaic', 'classical', 'roman', 'christian', 'jewish']

def try_to_get_from_freqdict(word, word_freqs_dict):
    total_N = sum([val for val in word_freqs_dict.values()])
    try:
        count = word_freqs_dict[word]
        freq = count #/ total_N
    except:
        freq = 0
    return freq

for sub in subcorpora:
    subset = LAGT[LAGT["subcorpus"]==sub]
    sents = [sent for work in subset["lemmatized_sentences"] for sent in work]
    word_freqs, words, vocabulary = get_vocab(sents)
    word_freqs_dict = dict(word_freqs)
    translated_df[sub + "_freq"] = translated_df["greek"].apply(lambda x: try_to_get_from_freqdict(x, word_freqs_dict))

In [103]:
translated_df

,raw_translation,greek,english_full,english_minimal,category,greek_transliterated,archaic_freq,classical_freq,roman_freq,christian_freq,jewish_freq
0,Αἰθίοψ - Ethiopian,Αἰθίοψ,Ethiopian,Ethiopian,,Aithíops,16,80,468,35,61
1,Δαναοί - Danaans,Δαναοί,Danaans,Danaans,,Danaoí,155,0,29,0,0
2,Δημήτηρ - Demeter,Δημήτηρ,Demeter,Demeter,,Dēmḗtēr,44,112,438,17,8
3,Διόνυσος - Dionysus,Διόνυσος,Dionysus,Dionysus,,Diónusos,16,168,892,84,15
4,Εὐθύφρων - Euthyphron,Εὐθύφρων,Euthyphron,Euthyphron,,Euthúphrōn,0,48,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
501,Ὠκεανός - Oceanus,Ὠκεανός,Oceanus,Oceanus,,Ōkeanós,70,53,505,24,17
502,"ῥέζω - govern, manage",ῥέζω,"govern, manage",govern,,rhézō,114,58,115,1,0
503,"ῥητορική - rhetoric, art of speaking",ῥητορική,"rhetoric, art of speaking",rhetoric,,rhētorikḗ,0,80,247,2,0
504,"ῥῆσις - speech, word",ῥῆσις,"speech, word",speech,,rhêsis,7,58,1177,11,23


In [104]:
religion_final = ["θεός", "Ζεύς", "εὐσεβής", 'ἱερός']
morality_final = ["ἀγαθός", "ἀρετή", "δίκαιος", "τιμή"]

In [106]:
translated_df.to_json("../data/translated_df_freqs.json")

In [127]:
translated_df[translated_df["greek"].isin(religion_final + morality_final)][["greek", "archaic_freq", "classical_freq", "roman_freq", "christian_freq", "jewish_freq"]]

,greek,archaic_freq,classical_freq,roman_freq,christian_freq,jewish_freq
5,Ζεύς,1322,2693,5821,1005,309
61,δίκαιος,79,3360,3163,1231,1115
101,εὐσεβής,11,99,189,87,93
126,θεός,1429,5050,11715,13922,9115
291,τιμή,129,800,2232,351,795
327,ἀγαθός,569,8934,16404,2034,2483
368,ἀρετή,107,2232,4370,894,1448


In [ ]:
translated_df[""]

# Preparing vocabularies

In [107]:
sents = [sent for work in LAGT["lemmatized_sentences"] for sent in work]

In [108]:
word_freqs, words, vocabulary = get_vocab(sents)

In [109]:
subcorpora = ['archaic', 'classical', 'roman', 'christian', 'jewish']

In [110]:
len(words)

14746005

In [111]:
len(vocabulary)

52252

In [112]:
subcorpora_vocabs = {}
for sub in subcorpora:
    subset = LAGT[LAGT["subcorpus"]==sub]
    sents = [sent for work in subset["lemmatized_sentences"] for sent in work]
    word_freqs, words, vocabulary = get_vocab(sents)
    vocabulary = [tup[0] for tup in word_freqs][:5000]
    subcorpora_vocabs[sub] = {
        "word_freqs" : word_freqs,
        "words" : words,
        "vocabulary" : vocabulary
    }

In [113]:
with open("../data/large_data/subcorpora_vocabs.pickle", "wb") as f:
    pickle.dump(subcorpora_vocabs, f)

In [114]:
subcorpora_vocabs["archaic"]["word_freqs"]

[('εἰμί', 3428),
 ('πᾶς', 1723),
 ('ἀνήρ', 1634),
 ('ἔρχομαι', 1623),
 ('λέγω', 1506),
 ('πολύς', 1499),
 ('θεός', 1429),
 ('ἔχω', 1412),
 ('Ζεύς', 1322),
 ('φημί', 1227),
 ('μέγας', 1171),
 ('ναῦς', 1129),
 ('φίλος', 1005),
 ('ὁράω', 936),
 ('ὅδε', 915),
 ('χείρ', 912),
 ('θυμός', 899),
 ('υἱός', 756),
 ('κακός', 753),
 ('γίγνομαι', 745),
 ('Ἀχαιός', 740),
 ('πατήρ', 729),
 ('Ὀδυσσεύς', 725),
 ('δίδωμι', 675),
 ('Τρώς', 649),
 ('παῖς', 626),
 ('γῆ', 618),
 ('αὐτός', 606),
 ('οὗτος', 605),
 ('ἄνθρωπος', 589),
 ('ἐμός', 586),
 ('φέρω', 580),
 ('ἵππος', 575),
 ('ἵστημι', 573),
 ('τίθημι', 569),
 ('ἀγαθός', 569),
 ('φρήν', 559),
 ('βάλλω', 554),
 ('ἔπος', 548),
 ('οἶδα', 546),
 ('αἱρέω', 533),
 ('μῦθος', 502),
 ('πρῶτος', 492),
 ('πόλις', 492),
 ('βαίνω', 484),
 ('ἐθέλω', 478),
 ('καλός', 474),
 ('ἄγω', 465),
 ('Ἕκτωρ', 456),
 ('ἑταῖρος', 452),
 ('γυνή', 439),
 ('τις', 439),
 ('δῖος', 434),
 ('ἔργον', 434),
 ('πούς', 416),
 ('Ἀχιλλεύς', 395),
 ('ἀθάνατος', 389),
 ('δῶμα', 387),
 ('μήτηρ',

In [115]:
religion_final = ["θεός", "Ζεύς", "εὐσεβής", 'ἱερός']
morality_final = ["ἀγαθός", "ἀρετή", "δίκαιος", "τιμή"]
keywords = religion_final + morality_final

In [139]:
print(", ".join(religion_final))

θεός, Ζεύς, εὐσεβής, ἱερός


In [140]:
print(", ".join(morality_final))


ἀγαθός, ἀρετή, δίκαιος, τιμή


In [136]:
keywords_freqs = []
keywords_counts = []
for keyword in keywords:
    row_data_freqs = {}
    row_data_counts = {}
    row_data_freqs["keyword"] = keyword
    row_data_counts["keyword"] = keyword
    for sub in subcorpora:
        if keyword in subcorpora_vocabs[sub]["words"]:
            word_freqs = subcorpora_vocabs[sub]["word_freqs"]
            wordcount = sum([t[1] for t in word_freqs])
            for tup in word_freqs:
                if tup[0] == keyword:
                    count = tup[1]
                    row_data_counts[sub + "_count"] = count
                    freq = np.round(count / wordcount, 5)
                    row_data_freqs[sub + "_freq"] = freq
    keywords_freqs.append(row_data_freqs)
    keywords_counts.append(row_data_counts)


In [137]:
pd.DataFrame(keywords_freqs)

,keyword,archaic_freq,classical_freq,roman_freq,christian_freq,jewish_freq
0,θεός,0.00730,0.00245,0.00207,0.01972,0.00953
1,Ζεύς,0.00676,0.00131,0.00103,0.00142,0.00032
2,εὐσεβής,0.00006,0.00005,0.00003,0.00012,0.00010
3,ἱερός,0.00079,0.00054,0.00058,0.00022,0.00098
4,ἀγαθός,0.00291,0.00434,0.00290,0.00288,0.00260
5,ἀρετή,0.00055,0.00108,0.00077,0.00127,0.00151
6,δίκαιος,0.00040,0.00163,0.00056,0.00174,0.00117
7,τιμή,0.00066,0.00039,0.00039,0.00050,0.00083


In [138]:
pd.DataFrame(keywords_counts)


,keyword,archaic_count,classical_count,roman_count,christian_count,jewish_count
0,θεός,1429,5050,11715,13922,9115
1,Ζεύς,1322,2693,5821,1005,309
2,εὐσεβής,11,99,189,87,93
3,ἱερός,155,1114,3278,153,933
4,ἀγαθός,569,8934,16404,2034,2483
5,ἀρετή,107,2232,4370,894,1448
6,δίκαιος,79,3360,3163,1231,1115
7,τιμή,129,800,2232,351,795


In [116]:
subcorpus_keywords = []
for sub in subcorpora:
    for keyword in keywords:
        if keyword in subcorpora_vocabs[sub]["words"]:
            word_freqs = subcorpora_vocabs[sub]["word_freqs"]
            wordcount = sum([t[1] for t in word_freqs])
            for tup in enumerate(word_freqs):
                if tup[1][0] == keyword:
                    rank = tup[0]
                    rank_ratio = np.round(rank / len(word_freqs), 5)
                    count = tup[1][1]
                    freq = np.round(count / wordcount, 5)
                    subcorpus_keywords.append([sub, keyword, rank, rank_ratio, count, freq])
                    break
        else:
            subcorpus_keywords.append([sub, keyword, None, None, None, None])

In [117]:
sub = "archaic"
word_freqs = subcorpora_vocabs[sub]["word_freqs"]
len(word_freqs)

18311

In [118]:
keyword = "εὐσεβής"
for tup in enumerate(word_freqs):
    if tup[1][0] == keyword:
        print(tup)

(2545, ('εὐσεβής', 11))


In [119]:
np.round(2545 / 18311, 5)

0.13899

In [129]:
subcorpus_keywords_df = pd.DataFrame(subcorpus_keywords, columns=["subcorpus", "keyword", "rank", "rank_ratio", "count", "freq"])
subcorpus_keywords_df

,subcorpus,keyword,rank,rank_ratio,count,freq
0,archaic,θεός,6,0.00033,1429,0.00730
1,archaic,Ζεύς,8,0.00044,1322,0.00676
2,archaic,εὐσεβής,2545,0.13899,11,0.00006
3,archaic,ἱερός,205,0.01120,155,0.00079
4,archaic,ἀγαθός,35,0.00191,569,0.00291
5,archaic,ἀρετή,306,0.01671,107,0.00055
6,archaic,δίκαιος,418,0.02283,79,0.00040
7,archaic,τιμή,247,0.01349,129,0.00066
8,classical,θεός,37,0.00051,5050,0.00245
9,classical,Ζεύς,89,0.00124,2693,0.00131


In [ ]:
subcorpus_keywords_df

In [121]:
len(subcorpora_vocabs["archaic"]["vocabulary"])

5000

In [122]:
shared_vocabulary = list((set(subcorpora_vocabs["archaic"]["vocabulary"])
    & set(subcorpora_vocabs["classical"]["vocabulary"])
    & set(subcorpora_vocabs["roman"]["vocabulary"])
    & set(subcorpora_vocabs["jewish"]["vocabulary"])
    & set(subcorpora_vocabs["christian"]["vocabulary"])))

In [123]:
len(shared_vocabulary)

1483

In [124]:
shared_vocabulary[:10]

['μεταβάλλω',
 'ἔτος',
 'ἔχω',
 'θῆλυς',
 'μάχαιρα',
 'ἀφοράω',
 'φλόξ',
 'γαστήρ',
 'ἐξελαύνω',
 'μείς']

In [125]:
with open("../data/shared_vocabulary.pickle", "wb") as f:
    pickle.dump(shared_vocabulary, f)